In [1]:
# ================================================
# Cell 1 NOTEBOOK 3 — LIVE TRADING ENGINE (Single Cell)
# ================================================
import os, glob, time, json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import gymnasium as gym
from typing import Dict, List, Optional
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
# ================================================
# Cell 2 - Paths (update if needed)
# ================================================
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")

LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

In [3]:
# ================================================
# Cell 3 - Configuration
# ================================================
DRY_RUN = False            # SET True for tests. Set False only after demo tests.
WINDOW = 50 # Minimum required lookback window for analysis (e.g., for indicators)
BUFFER = 60
COUNT = WINDOW + BUFFER
DEFAULT_LOT = 0.1
MAX_POS_PER_SYMBOL = 2
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3
TRAIL_PIPS_DEFAULT = 5


In [4]:
# ================================================
# Cell 4 - Timeframe
# ================================================
TIMEFRAME = "M15"
TF_MAP = {"M1": mt5.TIMEFRAME_M1, "M5": mt5.TIMEFRAME_M5, "M15": mt5.TIMEFRAME_M15,
          "M30": mt5.TIMEFRAME_M30, "H1": mt5.TIMEFRAME_H1, "H4": mt5.TIMEFRAME_H4, "D1": mt5.TIMEFRAME_D1}
TF_MT5 = TF_MAP.get(TIMEFRAME.upper(), mt5.TIMEFRAME_M15)


In [5]:
# ================================================
# Cell 5 - MT5 connection
# ================================================
def mt5_connect():
    if not mt5.initialize():
        # If already initialized, this returns False; check last_error for details
        err = mt5.last_error()
        print("mt5.initialize() returned False:", err)
        # try once more without parameters (assumes terminal open)
        try:
            ok = mt5.initialize()
            if not ok:
                raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")
        except Exception as e:
            raise RuntimeError("MT5 initialization failed: " + str(e))
    info = mt5.version()
    print("MT5 initialized. Version:", info)

def mt5_shutdown():
    try:
        mt5.shutdown()
    except Exception:
        pass


In [6]:
# ================================================
# Cell 6 - Symbols sellection
# ================================================
SYMBOLS: List[str] = [
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100","EURUSD"
]

In [7]:
# ================================================
# Cell 7 - Load PPO model
# ================================================
# Load PPO model
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Missing model file: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("Loaded PPO model:", MODEL_FILE)

# Load embeddings first so we can determine embed dim for dummy env
embeddings_raw = None
if os.path.exists(EMBED_FILE):
    try:
        # embeddings saved as array or dict; handle both
        emb_loaded = np.load(EMBED_FILE, allow_pickle=True)
        # if it's an array of embeddings, we'll handle mapping later
        embeddings_raw = emb_loaded
        print("Loaded raw embeddings from", EMBED_FILE)
    except Exception as e:
        print("Could not load embeddings:", e)
else:
    print("No embeddings file at", EMBED_FILE)

# infer embed dim
try:
    if isinstance(embeddings_raw, dict):
        EMBED_DIM = next(iter(embeddings_raw.values())).shape[0]
    else:
        # embeddings_raw could be ndarray shape (n,dim)
        EMBED_DIM = embeddings_raw.shape[1] if embeddings_raw is not None and embeddings_raw.ndim == 2 else 8
except Exception:
    EMBED_DIM = 8

# Build dummy env and load VecNormalize if exists
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        class DummyEnv(gym.Env):
            def __init__(self):
                super().__init__()
                obs_shape = (WINDOW, 5 + 1 + EMBED_DIM)  # 5 features + balance + embedding
                self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=obs_shape, dtype=np.float32)
                self.action_space = gym.spaces.Discrete(3)
            def reset(self, seed=None, options=None):
                return np.zeros(self.observation_space.shape, dtype=np.float32), {}
            def step(self, action):
                return np.zeros(self.observation_space.shape, dtype=np.float32), 0.0, True, False, {}
        venv = DummyVecEnv([lambda: DummyEnv()])
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("Loaded VecNormalize from", VEC_FILE)
    except Exception as e:
        print("Warning: VecNormalize.load failed:", e)
        vecnorm = None
else:
    print("No VecNormalize file found at", VEC_FILE)


Loaded PPO model: models\multiasset\ppo_multiasset.zip
Loaded raw embeddings from models\multiasset\asset_embeddings.npy
Loaded VecNormalize from models\multiasset\vec_normalize.pkl


In [8]:
# Cell 5
def load_scalers(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        # store Series so we can use .values if necessary
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_prepared_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
            if len(df) > window:
                datasets[safe] = df
        else:
            # attempt convert
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                tmp = tmp.dropna()
                if len(tmp) > window:
                    datasets[safe] = tmp
    return datasets

def load_embeddings(emb_file=EMBED_FILE, asset_map_file=ASSET_MAP_FILE, datasets_map=None):
    """
    Returns dict: safe_name -> embedding vector
    Handles either:
     - embeddings saved as ndarray (n_assets, embed_dim)
     - embeddings saved as dict-like (safe_name -> vector)
    """
    if not os.path.exists(emb_file):
        print("No embeddings file:", emb_file)
        return {}
    emb_loaded = np.load(emb_file, allow_pickle=True)
    # If it's a dict saved via np.save(..., allow_pickle=True) it may load as object
    if isinstance(emb_loaded, np.ndarray) and emb_loaded.dtype == object:
        # try to get item()
        try:
            emb_obj = emb_loaded.item()
            if isinstance(emb_obj, dict):
                return {k: np.array(v, dtype=np.float32) for k,v in emb_obj.items()}
        except Exception:
            pass
    # If embeddings is a plain ndarray (n, dim), map by asset_to_idx or by dataset order
    emb_dict = {}
    if os.path.exists(asset_map_file):
        try:
            am = pd.read_csv(asset_map_file, index_col=0, header=None).iloc[:,0].to_dict()
            # am: safe -> idx
            for safe, idx in am.items():
                idx = int(idx)
                if idx < emb_loaded.shape[0]:
                    emb_dict[safe] = np.array(emb_loaded[idx], dtype=np.float32)
        except Exception as e:
            print("asset_to_idx.csv read failed:", e)
    else:
        # fallback: map by prepared dataset order
        csvs = sorted(glob.glob(os.path.join(DATA_DIR, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb_loaded.shape[0]:
            for i,safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb_loaded[i], dtype=np.float32)
        else:
            print("Embeddings shape doesn't match CSV count; returning empty embedding mapping.")
    return emb_dict

scalers = load_scalers(DATA_DIR)
datasets = load_prepared_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, ASSET_MAP_FILE, datasets)

print("Loaded scalers:", len(scalers))
print("Loaded prepared datasets:", len(datasets))
print("Loaded embeddings:", len(embeddings))


Loaded scalers: 16
Loaded prepared datasets: 16
Loaded embeddings: 17


In [34]:
# ===================================== use this
# MT5 Live Data Fetching
# =====================================
def _fetch_single_symbol_bars(symbol: str, timeframe: int, count: int) -> Optional[pd.DataFrame]:
    info = mt5.symbol_info(symbol)
    if info is None:
        print(f"Symbol not found: {symbol}")
        return None

    if not info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"Could not activate symbol: {symbol}")
            return None

    lookback_days = max(1, count / 100) * 5
    start_time = datetime.now() - pd.Timedelta(days=lookback_days)

    bars = mt5.copy_rates_from(symbol, timeframe, start_time, count)
    if bars is None or len(bars) < WINDOW + 2:
        print(f"Not enough bars for {symbol}")
        return None

    df = pd.DataFrame(bars)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    if "tick_volume" in df.columns:
        df = df.rename(columns={"tick_volume": "volume"})

    df = df[["open","high","low","close","volume"]].copy()
    df["Close_raw"] = df["close"]

    return df

def fetch_symbol_bars_live(symbols: List[str], timeframe: int = TF_MT5, count: int = COUNT):
    out = {}
    if not mt5_connect():
        print("MT5 connection failed.")
        return out

    for sym in symbols:
        print("Fetching:", sym)
        df = _fetch_single_symbol_bars(sym, timeframe, count)
        if df is not None:
            out[sym] = df
            print(f"Fetched {len(df)} bars.")
        else:
            print("Failed:", sym)

    mt5_shutdown()
    return out

# =====================================
# Example Test
# =====================================
if __name__ == "__main__":
    subset = SYMBOLS[:5]
    data = fetch_symbol_bars_live(subset, timeframe=mt5.TIMEFRAME_H1, count=150)

    if data:
        s = list(data.keys())[0]
        print("\nSample:", s)
        print(data[s].tail())
    else:
        print("No data fetched.")


MT5 connected. Version: (500, 5430, '14 Nov 2025')
Fetching: Volatility 10 Index
Fetched 150 bars.
Fetching: Volatility 25 Index
Fetched 150 bars.
Fetching: Volatility 50 Index
Fetched 150 bars.
Fetching: Volatility 75 Index
Fetched 150 bars.
Fetching: Volatility 100 Index
Fetched 150 bars.

Sample: Volatility 10 Index
                         open      high       low     close  volume  Close_raw
time                                                                          
2025-11-17 20:00:00  5521.708  5529.371  5517.930  5522.794    1794   5522.794
2025-11-17 21:00:00  5522.746  5524.556  5516.647  5521.946    1797   5521.946
2025-11-17 22:00:00  5521.892  5524.662  5515.360  5523.347    1792   5523.347
2025-11-17 23:00:00  5523.441  5523.716  5516.775  5520.957    1791   5520.957
2025-11-18 00:00:00  5520.925  5527.380  5519.254  5524.247    1797   5524.247


In [35]:
# Cell 7
def build_obs_for_symbol(mt5_symbol, safe_name, window=WINDOW):
    """
    Returns obs (window, 5+1+embed_dim), vol_est, last_price
    """
    # Try live bars
    df_live = fetch_symbol_bars_live(mt5_symbol, TF_MT5, COUNT)
    if df_live is None:
        # fallback to prepared dataset if available
        if safe_name in datasets:
            df_pct = datasets[safe_name].iloc[-window:][['o_pc','h_pc','l_pc','c_pc','v_pc']]
            last_price = float(datasets[safe_name]['Close_raw'].iloc[-1])
        else:
            print("No data for", safe_name)
            return None, None, None
        features = df_pct.values.astype(np.float32)
        vol_est = float(df_pct[:,3].std())
    else:
        # compute pct change on live price bars and take last window
        df_pct = df_live[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            print("Not enough pct rows for", mt5_symbol)
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df_live['Close_raw'].iloc[-1])
        features = df_pct[['open','high','low','close','volume']].values.astype(np.float32)
        # map names to pct feature order later when scaling

    # scaler for safe_name must exist
    if safe_name not in scalers:
        print("Scaler missing for", safe_name)
        return None, None, None
    s = scalers[safe_name]
    # Ensure mean/std align with expected feature order o_pc,h_pc,l_pc,c_pc,v_pc
    mean = np.array(s['mean'].values if hasattr(s['mean'], "values") else s['mean'], dtype=np.float32)
    std  = np.array(s['std'].values if hasattr(s['std'], "values") else s['std'], dtype=np.float32)
    std = np.where(std==0, 1e-8, std)

    # If features are raw open/high/... we need to reorder to o_pc,h_pc,l_pc,c_pc,v_pc
    if features.shape[1] == 5 and np.all(np.isfinite(mean)):
        # If features were computed from live bars, they are in open,high,low,close,volume order
        # but scaler mean/std are for o_pc,h_pc,l_pc,c_pc,v_pc -> same order; proceed
        features_scaled = (features - mean) / std
    else:
        # fallback: try to use dataset values directly if shapes mismatch
        try:
            features_scaled = (features - mean) / std
        except Exception as e:
            print("Scaling error:", e)
            return None, None, None

    # embedding vector for safe_name
    if safe_name in embeddings:
        emb = np.array(embeddings[safe_name], dtype=np.float32)
    else:
        # fallback emb zero
        emb_dim = EMBED_DIM
        emb = np.zeros(emb_dim, dtype=np.float32)
    emb_block = np.repeat(emb.reshape(1,-1), window, axis=0)

    # balance column used in training (set to 1.0)
    balance_col = np.ones((window,1), dtype=np.float32)

    # Final observation: features_scaled (5) + balance (1) + embedding (emb_dim)
    obs = np.concatenate([features_scaled.astype(np.float32), balance_col, emb_block], axis=1)

    # apply VecNormalize if available (try safe approaches)
    if vecnorm is not None:
        try:
            # vecnorm may expect flattened obs; try normalize_obs if exists
            try:
                obs = vecnorm.normalize_obs(obs, False)
            except Exception:
                # fallback: use obs_rms stats if available
                mean_rms = getattr(vecnorm.obs_rms, "mean", None)
                var_rms = getattr(vecnorm.obs_rms, "var", None)
                flat = obs.reshape(1,-1)
                if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                    flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                    obs = flat_norm.reshape(obs.shape).astype(np.float32)
        except Exception as e:
            print("VecNormalize apply error:", e)

    vol_est = float(np.std(features[:,3]))  # std of c_pc (index 3 in open,high,low,close,volume)
    return obs.astype(np.float32), vol_est, last_price


In [36]:
# Cell 7
def build_obs_for_symbol(mt5_symbol, safe_name, window=WINDOW):
    """
    Returns obs (window, 5+1+embed_dim), vol_est, last_price
    """
    # Try live bars
    df_live = fetch_symbol_bars_live(mt5_symbol, TF_MT5, COUNT)
    if df_live is None:
        # fallback to prepared dataset if available
        if safe_name in datasets:
            df_pct = datasets[safe_name].iloc[-window:][['o_pc','h_pc','l_pc','c_pc','v_pc']]
            last_price = float(datasets[safe_name]['Close_raw'].iloc[-1])
        else:
            print("No data for", safe_name)
            return None, None, None
        features = df_pct.values.astype(np.float32)
        vol_est = float(df_pct[:,3].std())
    else:
        # compute pct change on live price bars and take last window
        df_pct = df_live[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            print("Not enough pct rows for", mt5_symbol)
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df_live['Close_raw'].iloc[-1])
        features = df_pct[['open','high','low','close','volume']].values.astype(np.float32)
        # map names to pct feature order later when scaling

    # scaler for safe_name must exist
    if safe_name not in scalers:
        print("Scaler missing for", safe_name)
        return None, None, None
    s = scalers[safe_name]
    # Ensure mean/std align with expected feature order o_pc,h_pc,l_pc,c_pc,v_pc
    mean = np.array(s['mean'].values if hasattr(s['mean'], "values") else s['mean'], dtype=np.float32)
    std  = np.array(s['std'].values if hasattr(s['std'], "values") else s['std'], dtype=np.float32)
    std = np.where(std==0, 1e-8, std)

    # If features are raw open/high/... we need to reorder to o_pc,h_pc,l_pc,c_pc,v_pc
    if features.shape[1] == 5 and np.all(np.isfinite(mean)):
        # If features were computed from live bars, they are in open,high,low,close,volume order
        # but scaler mean/std are for o_pc,h_pc,l_pc,c_pc,v_pc -> same order; proceed
        features_scaled = (features - mean) / std
    else:
        # fallback: try to use dataset values directly if shapes mismatch
        try:
            features_scaled = (features - mean) / std
        except Exception as e:
            print("Scaling error:", e)
            return None, None, None

    # embedding vector for safe_name
    if safe_name in embeddings:
        emb = np.array(embeddings[safe_name], dtype=np.float32)
    else:
        # fallback emb zero
        emb_dim = EMBED_DIM
        emb = np.zeros(emb_dim, dtype=np.float32)
    emb_block = np.repeat(emb.reshape(1,-1), window, axis=0)

    # balance column used in training (set to 1.0)
    balance_col = np.ones((window,1), dtype=np.float32)

    # Final observation: features_scaled (5) + balance (1) + embedding (emb_dim)
    obs = np.concatenate([features_scaled.astype(np.float32), balance_col, emb_block], axis=1)

    # apply VecNormalize if available (try safe approaches)
    if vecnorm is not None:
        try:
            # vecnorm may expect flattened obs; try normalize_obs if exists
            try:
                obs = vecnorm.normalize_obs(obs, False)
            except Exception:
                # fallback: use obs_rms stats if available
                mean_rms = getattr(vecnorm.obs_rms, "mean", None)
                var_rms = getattr(vecnorm.obs_rms, "var", None)
                flat = obs.reshape(1,-1)
                if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                    flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                    obs = flat_norm.reshape(obs.shape).astype(np.float32)
        except Exception as e:
            print("VecNormalize apply error:", e)

    vol_est = float(np.std(features[:,3]))  # std of c_pc (index 3 in open,high,low,close,volume)
    return obs.astype(np.float32), vol_est, last_price


In [37]:
# Cell 8
def pip_value(symbol):
    return 0.01 if "JPY" in symbol.upper() else 0.0001

def estimate_sl_pips(raw_symbol, last_price, vol_est, min_pips=5, max_pips=200):
    pip = pip_value(raw_symbol)
    abs_move = vol_est * last_price
    if abs_move <= 0:
        return DEFAULT_SL_PIPS_FALLBACK
    sl_raw = abs_move / pip
    sl_pips = int(np.clip(np.round(sl_raw * 1.5), min_pips, max_pips))
    return max(1, sl_pips)

def compute_sl_tp_prices(symbol, price, direction, sl_pips, tp_pips):
    pip = pip_value(symbol)
    if direction == "BUY":
        sl = price - sl_pips * pip
        tp = price + tp_pips * pip
    else:
        sl = price + sl_pips * pip
        tp = price - tp_pips * pip
    return float(sl), float(tp)

def get_positions_for_symbol(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_market_order(symbol, direction, lot, sl_price=None, tp_price=None):
    mt5_connect()
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ Market order cannot get tick for", symbol)
        return {"retcode": None, "comment": "NO_TICK"}
    price = float(tick.ask if direction=="BUY" else tick.bid)
    if DRY_RUN:
        # simulated
        return {"retcode":10009, "comment":"DRY_RUN", "price": price}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    res = mt5.order_send(req)
    return res

def close_position_by_ticket(ticket):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p,"type",None) == 0:
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [59]:
# Cell 9
def run_once_predict_and_manage(symbols_list):
    mt5_connect()
    acct = mt5.account_info()
    balance = float(acct.balance) if acct is not None else 10000.0

    header = not os.path.exists(LOG_FILE)

    for sym in symbols_list:
        safe_name = make_safe_name(sym)
        broker_sym = get_mt5_symbol_from_safe(safe_name)
        print(f"\n--- {broker_sym} (safe: {safe_name}) ---")

        obs_tuple = build_obs_for_symbol(broker_sym, safe_name, WINDOW)
        if obs_tuple is None:
            print("No observation for", safe_name)
            continue
        obs, vol, last_price = obs_tuple

        # model.predict expects batch dimension
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list,tuple,np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD,1=BUY,2=SELL)")

        positions = get_positions_for_symbol(broker_sym)
        print("Existing positions:", len(positions))

        # Auto-close opposing positions
        if a == 1:
            for p in positions:
                if getattr(p,"type",None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:
            for p in positions:
                if getattr(p,"type",None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # Refresh positions
        positions = get_positions_for_symbol(broker_sym)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", broker_sym)
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a==1 else "SELL"
                lot = DEFAULT_LOT
                sl_pips = estimate_sl_pips(broker_sym, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                sl_price, tp_price = compute_sl_tp_prices(broker_sym, last_price, direction, sl_pips, tp_pips)

                res = place_market_order(broker_sym, direction, lot, sl_price, tp_price)
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res,"retcode",None)
                    comment = getattr(res,"comment","")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": broker_sym,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        # Trailing SL for existing positions
        for p in get_positions_for_symbol(broker_sym):
            trail_pips = max(5, int(estimate_sl_pips(broker_sym, last_price, vol) // 2))
            pip = pip_value(broker_sym)
            if getattr(p,"type",None) == 0:  # BUY
                cur_price = mt5.symbol_info_tick(broker_sym).bid
                new_sl = float(cur_price - trail_pips * pip)
            else:
                cur_price = mt5.symbol_info_tick(broker_sym).ask
                new_sl = float(cur_price + trail_pips * pip)

            if DRY_RUN:
                print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
            else:
                req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": broker_sym, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                r = mt5.order_send(req)
                print("Modify SL result:", getattr(r,"retcode",None))

    mt5_shutdown()
    print("\nSingle pass complete.")


In [60]:
# Cell 10 — Run
# Use a subset first to test quickly
test_list = SYMBOLS  # or SYMBOLS[:5]
run_once_predict_and_manage(test_list)


Mock: Connecting to MetaTrader5...

--- VOLATILITY 10 INDEX (safe: volatility 10 index) ---


TypeError: build_obs_for_symbol() takes 2 positional arguments but 3 were given

In [61]:
# ================================================
# NOTEBOOK 3 — LIVE TRADING ENGINE (Single Cell) partially okay
# ================================================
import os, glob, time, json
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import gymnasium as gym
from typing import Dict, List, Optional

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# ---------- Config ----------
DRY_RUN = False
WINDOW = 50
BUFFER = 60
COUNT = WINDOW + BUFFER

DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")

TIMEFRAME = "M15"
TF_MAP = {
    "M1": mt5.TIMEFRAME_M1, "M5": mt5.TIMEFRAME_M5, "M15": mt5.TIMEFRAME_M15,
    "M30": mt5.TIMEFRAME_M30, "H1": mt5.TIMEFRAME_H1, "H4": mt5.TIMEFRAME_H4, "D1": mt5.TIMEFRAME_D1
}
TF_MT5 = TF_MAP.get(TIMEFRAME.upper(), mt5.TIMEFRAME_M15)

DEFAULT_LOT = 0.1
MAX_POS_PER_SYMBOL = 2
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3
TRAIL_PIPS_DEFAULT = 5

LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

print("Notebook 3 config loaded. DRY_RUN =", DRY_RUN)

# =====================================
# MT5 Connection Utilities
# =====================================
def mt5_connect() -> bool:
    if not mt5.initialize():
        print("MT5 initialize() failed:", mt5.last_error())
        return False
    print("MT5 connected.")
    return True

def mt5_shutdown():
    try:
        mt5.shutdown()
    except:
        pass

# =====================================
# Symbol List
# =====================================
SYMBOLS = [
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100","EURUSD"
]

# =====================================
# Load PPO Model
# =====================================
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Missing model file: " + MODEL_FILE)

model = PPO.load(MODEL_FILE)
print("Loaded PPO model:", MODEL_FILE)

# =====================================
# Embeddings
# =====================================
embeddings_raw = None
if os.path.exists(EMBED_FILE):
    emb_loaded = np.load(EMBED_FILE, allow_pickle=True)
    embeddings_raw = emb_loaded
    print("Loaded embeddings")
else:
    print("No embeddings file found")

try:
    if isinstance(embeddings_raw, dict):
        EMBED_DIM = next(iter(embeddings_raw.values())).shape[0]
    else:
        EMBED_DIM = embeddings_raw.shape[1] if embeddings_raw is not None else 8
except:
    EMBED_DIM = 8

# =====================================
# VecNormalize Loader
# =====================================
vecnorm = None

if os.path.exists(VEC_FILE):
    class DummyEnv(gym.Env):
        def __init__(self):
            super().__init__()
            obs_shape = (WINDOW, 5 + 1 + EMBED_DIM)
            self.observation_space = gym.spaces.Box(
                low=-np.inf, high=np.inf, shape=obs_shape, dtype=np.float32
            )
            self.action_space = gym.spaces.Discrete(3)

        def reset(self, seed=None, options=None):
            return np.zeros(self.observation_space.shape, dtype=np.float32), {}

        def step(self, action):
            return np.zeros(self.observation_space.shape, dtype=np.float32), 0.0, True, False, {}

    venv = DummyVecEnv([lambda: DummyEnv()])
    vecnorm = VecNormalize.load(VEC_FILE, venv)
    vecnorm.training = False
    vecnorm.norm_reward = False
    print("Loaded VecNormalize")
else:
    print("No VecNormalize file.")

# =====================================
# Data Loading Utilities
# =====================================
def load_scalers(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_prepared_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
            if len(df) > window:
                datasets[safe] = df
    return datasets

def load_embeddings(emb_file=EMBED_FILE, asset_map_file=ASSET_MAP_FILE):
    if not os.path.exists(emb_file):
        return {}

    emb_loaded = np.load(emb_file, allow_pickle=True)

    if isinstance(emb_loaded, np.ndarray) and emb_loaded.dtype == object:
        try:
            dct = emb_loaded.item()
            if isinstance(dct, dict):
                return {k: np.array(v, dtype=np.float32) for k,v in dct.items()}
        except:
            pass

    emb_dict = {}
    if os.path.exists(asset_map_file):
        am = pd.read_csv(asset_map_file, index_col=0, header=None).iloc[:,0].to_dict()
        for safe, idx in am.items():
            emb_dict[safe] = np.array(emb_loaded[int(idx)], dtype=np.float32)

    return emb_dict

scalers = load_scalers()
datasets = load_prepared_datasets()
embeddings = load_embeddings()

print("Loaded scalers:", len(scalers))
print("Loaded datasets:", len(datasets))
print("Loaded embeddings:", len(embeddings))

# =====================================
# MT5 Live Data Fetching
# =====================================
def _fetch_single_symbol_bars(symbol: str, timeframe: int, count: int) -> Optional[pd.DataFrame]:
    info = mt5.symbol_info(symbol)
    if info is None:
        print(f"Symbol not found: {symbol}")
        return None

    if not info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"Could not activate symbol: {symbol}")
            return None

    lookback_days = max(1, count / 100) * 5
    start_time = datetime.now() - pd.Timedelta(days=lookback_days)

    bars = mt5.copy_rates_from(symbol, timeframe, start_time, count)
    if bars is None or len(bars) < WINDOW + 2:
        print(f"Not enough bars for {symbol}")
        return None

    df = pd.DataFrame(bars)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    if "tick_volume" in df.columns:
        df = df.rename(columns={"tick_volume": "volume"})

    df = df[["open","high","low","close","volume"]].copy()
    df["Close_raw"] = df["close"]

    return df

def fetch_symbol_bars_live(symbols: List[str], timeframe: int = TF_MT5, count: int = COUNT):
    out = {}
    if not mt5_connect():
        print("MT5 connection failed.")
        return out

    for sym in symbols:
        print("Fetching:", sym)
        df = _fetch_single_symbol_bars(sym, timeframe, count)
        if df is not None:
            out[sym] = df
            print(f"Fetched {len(df)} bars.")
        else:
            print("Failed:", sym)

    mt5_shutdown()
    return out

# =====================================
# Example Test
# =====================================
if __name__ == "__main__":
    subset = SYMBOLS[:5]
    data = fetch_symbol_bars_live(subset, timeframe=mt5.TIMEFRAME_H1, count=150)

    if data:
        s = list(data.keys())[0]
        print("\nSample:", s)
        print(data[s].tail())
    else:
        print("No data fetched.")


Notebook 3 config loaded. DRY_RUN = False
Loaded PPO model: models\multiasset\ppo_multiasset.zip
Loaded embeddings
Loaded VecNormalize
Loaded scalers: 16
Loaded datasets: 16
Loaded embeddings: 17
MT5 connected.
Fetching: Volatility 10 Index
Fetched 150 bars.
Fetching: Volatility 25 Index
Fetched 150 bars.
Fetching: Volatility 50 Index
Fetched 150 bars.
Fetching: Volatility 75 Index
Fetched 150 bars.
Fetching: Volatility 100 Index
Fetched 150 bars.

Sample: Volatility 10 Index
                         open      high       low     close  volume  Close_raw
time                                                                          
2025-11-17 21:00:00  5522.746  5524.556  5516.647  5521.946    1797   5521.946
2025-11-17 22:00:00  5521.892  5524.662  5515.360  5523.347    1792   5523.347
2025-11-17 23:00:00  5523.441  5523.716  5516.775  5520.957    1791   5520.957
2025-11-18 00:00:00  5520.925  5527.380  5519.254  5524.247    1797   5524.247
2025-11-18 01:00:00  5524.197  5535.667  5524

In [65]:
# ================= Full Trade Execution — Single Notebook Cell =================
# Copy-paste this whole block into one Jupyter cell and run.
import os, glob, time, json, traceback
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import gymnasium as gym
from typing import Dict, List, Optional
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# ---------- USER CONFIG ----------
DRY_RUN = False                # SAFE DEFAULT: set False only after demo testing
WINDOW = 50                   # MUST match training
BUFFER = 60
COUNT = WINDOW + BUFFER
TIMEFRAME = "M15"             # M1, M5, M15, M30, H1, H4, D1 (must match training timeframe)
TF_MAP = {"M1": mt5.TIMEFRAME_M1, "M5": mt5.TIMEFRAME_M5, "M15": mt5.TIMEFRAME_M15,
          "M30": mt5.TIMEFRAME_M30, "H1": mt5.TIMEFRAME_H1, "H4": mt5.TIMEFRAME_H4, "D1": mt5.TIMEFRAME_D1}
TF_MT5 = TF_MAP.get(TIMEFRAME.upper(), mt5.TIMEFRAME_M15)

DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")

DEFAULT_LOT = 0.1
MAX_POS_PER_SYMBOL = 2
TP_MULT = 3
DEFAULT_SL_PIPS_FALLBACK = 20
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")

# Symbols you want the robot to consider (broker names may differ — use MANUAL_SYMBOL_MAP)
SYMBOLS = [
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100","EURUSD"
]

# If broker uses different symbol names, map safe_name -> broker_symbol
MANUAL_SYMBOL_MAP: Dict[str, str] = {
    # example: "Volatility_75_Index": "DerivVol75"
    # add entries if MT5 symbol_info(name) would be None for the plain name
}

# ----------------- Helpers -----------------
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

def get_mt5_symbol_from_safe(safe_name: str) -> str:
    return MANUAL_SYMBOL_MAP.get(safe_name, safe_name.replace("_", " "))

def mt5_connect() -> bool:
    try:
        if not mt5.initialize():
            # try again to get more helpful info
            err = mt5.last_error()
            print("mt5.initialize() first attempt returned False:", err)
            ok = mt5.initialize()
            if not ok:
                print("mt5.initialize() retry failed:", mt5.last_error())
                return False
        print("MT5 connected. Version:", mt5.version())
        return True
    except Exception as e:
        print("MT5 connection error:", e)
        return False

def mt5_shutdown():
    try:
        mt5.shutdown()
    except Exception:
        pass

# ----------------- Load model, VecNormalize, embeddings, scalers, datasets -----------------
print("Loading model and preprocessors...")
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model file missing: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("PPO model loaded:", MODEL_FILE)

# load embeddings (attempt different formats)
embeddings = {}
EMBED_DIM = 8
if os.path.exists(EMBED_FILE):
    try:
        emb_loaded = np.load(EMBED_FILE, allow_pickle=True)
        # if saved dict
        if isinstance(emb_loaded, np.ndarray) and emb_loaded.dtype == object:
            try:
                obj = emb_loaded.item()
                if isinstance(obj, dict):
                    embeddings = {k: np.array(v, dtype=np.float32) for k, v in obj.items()}
            except Exception:
                pass
        if len(embeddings) == 0:
            # if array mapping by index
            try:
                arr = np.array(emb_loaded)
                if arr.ndim == 2:
                    # attempt mapping with asset_to_idx
                    if os.path.exists(ASSET_MAP_FILE):
                        try:
                            am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
                            for safe, idx in am.items():
                                idx = int(idx)
                                if idx < arr.shape[0]:
                                    embeddings[safe] = np.array(arr[idx], dtype=np.float32)
                        except Exception:
                            pass
                # final fallback: if embeddings length matches input CSV count, map by CSV order in data dir
                if len(embeddings) == 0 and arr.ndim == 2:
                    csvs = sorted(glob.glob(os.path.join(DATA_DIR, "*_normalized.csv")))
                    safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
                    if len(safe_list) == arr.shape[0]:
                        for i, s in enumerate(safe_list):
                            embeddings[s] = np.array(arr[i], dtype=np.float32)
            except Exception:
                pass
    except Exception as e:
        print("Failed to load embeddings:", e)
if len(embeddings) > 0:
    EMBED_DIM = next(iter(embeddings.values())).shape[0]
print("Embeddings loaded:", len(embeddings), "embed_dim:", EMBED_DIM)

# VecNormalize (safe load using dummy env)
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        class _DummyEnv(gym.Env):
            def __init__(self):
                super().__init__()
                self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, 5 + 1 + EMBED_DIM), dtype=np.float32)
                self.action_space = gym.spaces.Discrete(3)
            def reset(self, seed=None, options=None):
                return np.zeros(self.observation_space.shape, dtype=np.float32), {}
            def step(self, action):
                return np.zeros(self.observation_space.shape, dtype=np.float32), 0.0, True, False, {}
        venv = DummyVecEnv([lambda: _DummyEnv()])
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("VecNormalize loaded:", VEC_FILE)
    except Exception as e:
        print("VecNormalize load failed (continuing without it):", e)
        vecnorm = None
else:
    print("No VecNormalize file found; continuing without it.")

# scalers
def load_scalers(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

scalers = load_scalers()
print("Scalers loaded:", len(scalers))

# datasets (prepared normalized CSVs)
def load_prepared_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            tmp = df[expected].dropna()
            if len(tmp) > window:
                datasets[safe] = tmp
        else:
            # try convert from OHLCV if possible
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                tmp = tmp.dropna()
                if len(tmp) > window:
                    datasets[safe] = tmp
    return datasets

datasets = load_prepared_datasets()
print("Prepared datasets loaded:", len(datasets))

# ----------------- Live bars fetch -----------------
def _fetch_single_symbol_bars(mt5_symbol: str, timeframe: int, count: int) -> Optional[pd.DataFrame]:
    info = mt5.symbol_info(mt5_symbol)
    if info is None:
        # no such symbol in terminal
        return None
    if not info.visible:
        try:
            mt5.symbol_select(mt5_symbol, True)
        except Exception:
            pass
    end_time = datetime.now()
    bars = mt5.copy_rates_from(mt5_symbol, timeframe, end_time, count)
    if bars is None or len(bars) < WINDOW + 2:
        return None
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    if 'tick_volume' in df.columns:
        df = df.rename(columns={'tick_volume':'volume'})
    df = df[['open','high','low','close','volume']].copy()
    df['Close_raw'] = df['close']
    return df

# ----------------- Observation builder -----------------
def build_obs_for_symbol(mt5_symbol: str, safe_name: str, window: int = WINDOW):
    # try live bars
    df_live = _fetch_single_symbol_bars(mt5_symbol, TF_MT5, COUNT)
    if df_live is None:
        # fallback to prepared dataset
        if safe_name not in datasets:
            return None, None, None
        df_pct = datasets[safe_name].iloc[-window:][['o_pc','h_pc','l_pc','c_pc','v_pc']]
        last_price = float(datasets[safe_name]['Close_raw'].iloc[-1])
        features = df_pct.values.astype(np.float32)
    else:
        # compute percent changes from live OHLCV
        df_pct = df_live[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df_live['Close_raw'].iloc[-1])
        features = df_pct[['open','high','low','close','volume']].values.astype(np.float32)

    # scaler required
    if safe_name not in scalers:
        return None, None, None
    s = scalers[safe_name]
    mean = np.array(s['mean'].values if hasattr(s['mean'],'values') else s['mean'], dtype=np.float32)
    std = np.array(s['std'].values if hasattr(s['std'],'values') else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)

    if features.shape[1] != len(mean):
        return None, None, None

    features_scaled = (features - mean) / std

    # embedding
    emb_vec = embeddings.get(safe_name, np.zeros(EMBED_DIM, dtype=np.float32))
    emb_block = np.repeat(emb_vec.reshape(1,-1), window, axis=0)

    # balance norm column (training used some balance column — here we set 1.0)
    balance_col = np.ones((window,1), dtype=np.float32)

    obs = np.concatenate([features_scaled.astype(np.float32), balance_col, emb_block], axis=1)

    # apply VecNormalize if available (best-effort)
    if vecnorm is not None:
        try:
            # prefer normalize_obs if available
            try:
                obs = vecnorm.normalize_obs(obs, False)
            except Exception:
                # fallback to obs_rms stats
                flat = obs.reshape(1,-1)
                mean_rms = getattr(vecnorm.obs_rms, "mean", None)
                var_rms = getattr(vecnorm.obs_rms, "var", None)
                if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                    flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                    obs = flat_norm.reshape(obs.shape).astype(np.float32)
        except Exception:
            pass

    vol_est = float(np.std(features[:,3]))  # std of close pct
    return obs.astype(np.float32), vol_est, last_price

# ----------------- Trading helpers -----------------
def pip_value(symbol: str) -> float:
    return 0.01 if "JPY" in symbol.upper() else 0.0001

def estimate_sl_pips(symbol: str, last_price: float, vol_est: float, min_pips=5, max_pips=200) -> int:
    pip = pip_value(symbol)
    abs_move = vol_est * last_price
    if abs_move <= 0:
        return DEFAULT_SL_PIPS_FALLBACK
    sl_raw = abs_move / pip
    sl_pips = int(np.clip(np.round(sl_raw * 1.5), min_pips, max_pips))
    return max(1, sl_pips)

def compute_sl_tp_prices(symbol: str, price: float, direction: str, sl_pips: int, tp_pips: int):
    pip = pip_value(symbol)
    if direction == "BUY":
        sl = price - sl_pips * pip
        tp = price + tp_pips * pip
    else:
        sl = price + sl_pips * pip
        tp = price - tp_pips * pip
    return float(sl), float(tp)

def get_positions_for_symbol(symbol: str):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_market_order(symbol: str, direction: str, lot: float, sl_price: Optional[float]=None, tp_price: Optional[float]=None):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return {"retcode": None, "comment": "NO_TICK"}
    price = float(tick.ask if direction == "BUY" else tick.bid)
    if DRY_RUN:
        return {"retcode": 10009, "comment": "DRY_RUN", "price": price}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    return mt5.order_send(req)

def close_position_by_ticket(ticket: int):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p,"type",None) == 0:  # BUY
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)

# ----------------- Main run loop: single pass -----------------
def run_once_predict_and_manage(symbols_list: List[str]):
    if not mt5_connect():
        print("MT5 connect failed — aborting.")
        return

    acct = mt5.account_info()
    balance = float(acct.balance) if acct is not None else 10000.0
    header = not os.path.exists(LOG_FILE)

    for raw_sym in symbols_list:
        safe_name = make_safe_name(raw_sym)
        broker_sym = get_mt5_symbol_from_safe(safe_name)
        print(f"\n=== {broker_sym} (safe: {safe_name}) ===")

        obs_tuple = build_obs_for_symbol(broker_sym, safe_name, WINDOW)
        if obs_tuple is None:
            print("No observation for", safe_name, "- skipping")
            continue
        obs, vol, last_price = obs_tuple

        # predict
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list, tuple, np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, "(0=HOLD,1=BUY,2=SELL)")

        # manage positions
        positions = get_positions_for_symbol(broker_sym)
        print("Existing positions:", len(positions))

        # auto-close opposing positions
        if a == 1:  # BUY signal -> close SELL positions
            for p in positions:
                if getattr(p, "type", None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:  # SELL signal -> close BUY positions
            for p in positions:
                if getattr(p, "type", None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh
        positions = get_positions_for_symbol(broker_sym)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", broker_sym, "- skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a == 1 else "SELL"
                lot = DEFAULT_LOT
                sl_pips = estimate_sl_pips(broker_sym, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                sl_price, tp_price = compute_sl_tp_prices(broker_sym, last_price, direction, sl_pips, tp_pips)

                res = place_market_order(broker_sym, direction, lot, sl_price, tp_price)
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": broker_sym,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        # trailing SL pass for current positions
        for p in get_positions_for_symbol(broker_sym):
            trail_pips = max(5, int(estimate_sl_pips(broker_sym, last_price, vol) // 2))
            pip = pip_value(broker_sym)
            if getattr(p, "type", None) == 0:  # BUY
                cur_price = mt5.symbol_info_tick(broker_sym).bid
                new_sl = float(cur_price - trail_pips * pip)
            else:
                cur_price = mt5.symbol_info_tick(broker_sym).ask
                new_sl = float(cur_price + trail_pips * pip)

            if DRY_RUN:
                print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
            else:
                try:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": broker_sym, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL retcode:", getattr(r, "retcode", None))
                except Exception as e:
                    print("Modify SL failed:", e)

    mt5_shutdown()
    print("\nSingle pass complete. Logs written to:", LOG_FILE)

# ------------- run once (example) -------------
# For safety, you can test a subset:
test_symbols = SYMBOLS[:]  # change as needed
run_once_predict_and_manage(test_symbols)

# ------------- Optional continuous loop -------------
# Uncomment and use with caution (ensure DRY_RUN=False only after demo verification)
try:
    while True:
        run_once_predict_and_manage(SYMBOLS)
        time.sleep(60)   # run each minute for M1/M15 choose appropriate sleep
except KeyboardInterrupt:
    print("Stopped by user.")


Loading model and preprocessors...
PPO model loaded: models\multiasset\ppo_multiasset.zip
Embeddings loaded: 17 embed_dim: 8
VecNormalize loaded: models\multiasset\vec_normalize.pkl
Scalers loaded: 16
Prepared datasets loaded: 16
MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0


C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12

Placed SELL lot 0.1 retcode 10009

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode 10009

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 75 Index (safe: Volatility_75_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 100 Index (safe: Volatility_100_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 10 1s Index (safe: Volatility_10_1s_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode None

=== Volatility 25 1s Index (safe: Volatility_25_1s_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 50 1s Index (safe: Volatility_50_1s_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 75 1s

C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12

Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 75 Index (safe: Volatility_75_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 100 Index (safe: Volatility_100_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 10 1s Index (safe: Volatility_10_1s_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode None

=== Volatility 25 1s Index (safe: Volatility_25_1s_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 50 1s Index (safe: Volatility_50_1s_Index) ===
Signal: 0 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD

=== Volatility 75 1s Index (safe: Volatility_75_1s_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode None

=== Volatility 100 1s Index (safe: Volatility_100_1s_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing po

...

MT5 connected. Version: (500, 5430, '14 Nov 2025')

=== Volatility 10 Index (safe: Volatility_10_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode 10009

=== Volatility 25 Index (safe: Volatility_25_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 50 Index (safe: Volatility_50_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 75 Index (safe: Volatility_75_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 100 Index (safe: Volatility_100_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Volatility 10 1s Index (safe: Volatility_10_1s_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode None

=== Volatility 25 1s Index (safe: Volatility_25_1s_Index) ===
Signal: 0 (0=

C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12

0
Placed SELL lot 0.1 retcode 10009

=== Jump 25 Index (safe: Jump_25_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Jump 50 Index (safe: Jump_50_Index) ===
Signal: 1 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed BUY lot 0.1 retcode 10009

=== Jump 75 Index (safe: Jump_75_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode 10009

=== Jump 100 Index (safe: Jump_100_Index) ===
Signal: 2 (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
Placed SELL lot 0.1 retcode 10009

=== Step Index 25 (safe: Step_Index_25) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 50 (safe: Step_Index_50) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 75 (safe: Step_Index_75) ===
Prediction error: 'NoneType' object is not subscriptable

=== Step Index 100 (safe: Step_Index_100) ===
Prediction error: 'NoneType' object is not subscriptable

=== EURUSD (safe: EUR

C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12336\2601678793.py:432: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_12

Stopped by user.


In [68]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime

# --- Configuration ---
SYMBOL = "EURUSD"          # The symbol you wish to trade
VOLUME = 0.01              # The volume of the trade (in lots)
DEVIATION = 10             # Maximum acceptable price deviation from the requested price
MAGIC = 20250615           # A unique identifier for your trades
COMMENT = "Jupyter_Trade_Bot"
SL_OFFSET_PIPS = 30        # Stop Loss offset in pips (e.g., 30 pips)
TP_OFFSET_PIPS = 60        # Take Profit offset in pips (e.g., 60 pips)

def connect_mt5(login=None, password=None, server=None):
    """Initializes and connects to the MT5 terminal."""
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # If credentials are provided, try to log in (useful if terminal is not running)
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
        
    print(f"Successfully connected to MT5 terminal.")
    print(f"Account: {mt5.account_info().login}")
    return True

def get_symbol_info(symbol):
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"{symbol} not found.")
        return None, None

    if not symbol_info.visible:
        print(f"{symbol} is not visible in Market Watch. Attempting to add it...")
        if not mt5.symbol_select(symbol, True):
            print(f"Could not select {symbol}. Trading is not possible.")
            return None, None

    # Get the tick size (e.g., 0.00001 for 5-digit brokers)
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Could not get tick info for {symbol}.")
        return None, None

    # Calculate the value of a single pip for calculations
    # This is typically the 'point' value of the symbol.
    pip_value = symbol_info.point
    
    print(f"\n--- Symbol Information for {symbol} ---")
    print(f"Digits: {symbol_info.digits}")
    print(f"Tick Size (Point): {symbol_info.point}")
    print(print(f"Bid: {tick.bid} | Ask: {tick.ask}"))
    
    return symbol_info, pip_value


def place_market_order(symbol_info, pip_value, order_type, volume):
    """
    Constructs and sends a market order (BUY or SELL).
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    :param volume: Lot size
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        return None

    # Determine execution price and trade type
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS * pip_value)
        tp_price = price + (TP_OFFSET_PIPS * pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS * pip_value)
        tp_price = price - (TP_OFFSET_PIPS * pip_value)
    else:
        print("Invalid order type.")
        return None

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": round(sl_price, symbol_info.digits), # Use symbol digits for rounding
        "tp": round(tp_price, symbol_info.digits), # Use symbol digits for rounding
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, # Good 'Till Cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }

    # Send order to MT5
    print(f"\nSending {type_text} order for {symbol} at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request


def format_result(result, request):
    """Formats the order result and request into a clean pandas DataFrame."""
    if result is None:
        return pd.DataFrame([{"Status": "Failed to send order (local error)"}])
        
    # Create a dictionary for the output table
    output = {
        #"Time": datetime.fromtimestamp(result.time).strftime('%Y-%m-%d %H:%M:%S'),
        "Time": datetime.fromtimestamp(result.time).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": request.get("symbol"),
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        "Price": f"{result.price:.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "SL": f"{request.get('sl'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "TP": f"{request.get('tp'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
    }
    
    # Check if the result includes the position ticket
    if hasattr(result, 'position'):
        output["Position Ticket"] = result.position
    
    return pd.DataFrame([output])


# ==============================================================================
# --- EXECUTION BLOCK ---
# ==============================================================================

# 1. Connect to MT5
if connect_mt5():
    # 2. Get Symbol Information
    symbol_info, pip_value = get_symbol_info(SYMBOL)

    if symbol_info and pip_value:
        # --- PLACE A TRADE HERE ---
        # NOTE: Use mt5.ORDER_TYPE_BUY for a buy order, mt5.ORDER_TYPE_SELL for a sell order.
        
        # Example 1: Place a BUY market order
        trade_result, trade_request = place_market_order(
            symbol_info, 
            pip_value, 
            mt5.ORDER_TYPE_BUY, 
            VOLUME
        )

        # 3. Display the result in a clean table (ideal for Jupyter)
        if trade_result:
            result_df = format_result(trade_result, trade_request)
            print("\n" + "="*50)
            print("--- TRADE EXECUTION RESULT ---")
            print("="*50)
            print(result_df.to_markdown(index=False)) # Display as markdown table in Jupyter
        
        # You can place another trade here if needed, for example a SELL
        time.sleep(2)
        trade_result_sell, trade_request_sell = place_market_order(symbol_info, pip_value, mt5.ORDER_TYPE_SELL, VOLUME)
        if trade_result_sell:
            result_df_sell = format_result(trade_result_sell, trade_request_sell)
            print("\n--- SECOND TRADE RESULT ---")
            print(result_df_sell.to_markdown(index=False))
        

    # 4. Shutdown MT5 connection
    mt5.shutdown()
    print("\nMT5 connection successfully shut down.")

Successfully connected to MT5 terminal.
Account: 40866995

--- Symbol Information for EURUSD ---
Digits: 5
Tick Size (Point): 1e-05
Bid: 1.15485 | Ask: 1.15496
None

Sending BUY order for EURUSD at 1.15496...


AttributeError: 'OrderSendResult' object has no attribute 'time'

In [69]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime

# --- Configuration ---
SYMBOL = "EURUSD"          # The symbol you wish to trade
VOLUME = 0.01              # The volume of the trade (in lots)
DEVIATION = 10             # Maximum acceptable price deviation from the requested price
MAGIC = 20250615           # A unique identifier for your trades
COMMENT = "Jupyter_Trade_Bot"
SL_OFFSET_PIPS = 30        # Stop Loss offset in pips (e.g., 30 pips)
TP_OFFSET_PIPS = 60        # Take Profit offset in pips (e.g., 60 pips)

def connect_mt5(login=None, password=None, server=None):
    """Initializes and connects to the MT5 terminal."""
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # If credentials are provided, try to log in (useful if terminal is not running)
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
        
    print(f"Successfully connected to MT5 terminal.")
    print(f"Account: {mt5.account_info().login}")
    return True

def get_symbol_info(symbol):
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"{symbol} not found.")
        return None, None

    if not symbol_info.visible:
        print(f"{symbol} is not visible in Market Watch. Attempting to add it...")
        if not mt5.symbol_select(symbol, True):
            print(f"Could not select {symbol}. Trading is not possible.")
            return None, None

    # Get the tick size (e.g., 0.00001 for 5-digit brokers)
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Could not get tick info for {symbol}.")
        return None, None

    # Calculate the value of a single pip for calculations
    # This is typically the 'point' value of the symbol.
    pip_value = symbol_info.point
    
    print(f"\n--- Symbol Information for {symbol} ---")
    print(f"Digits: {symbol_info.digits}")
    print(f"Tick Size (Point): {symbol_info.point}")
    print(print(f"Bid: {tick.bid} | Ask: {tick.ask}"))
    
    return symbol_info, pip_value


def place_market_order(symbol_info, pip_value, order_type, volume):
    """
    Constructs and sends a market order (BUY or SELL).
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    :param volume: Lot size
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        return None

    # Determine execution price and trade type
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS * pip_value)
        tp_price = price + (TP_OFFSET_PIPS * pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS * pip_value)
        tp_price = price - (TP_OFFSET_PIPS * pip_value)
    else:
        print("Invalid order type.")
        return None

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": round(sl_price, symbol_info.digits), # Use symbol digits for rounding
        "tp": round(tp_price, symbol_info.digits), # Use symbol digits for rounding
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, # Good 'Till Cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }

    # Send order to MT5
    print(f"\nSending {type_text} order for {symbol} at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request


def format_result(result, request):
    """Formats the order result and request into a clean pandas DataFrame."""
    if result is None:
        return pd.DataFrame([{"Status": "Failed to send order (local error)"}])
    
    # FIX: Use result.time_done for the execution time, as result.time is often not available
    # for the immediate OrderSendResult object. Fall back to local time if time_done is zero.
    timestamp = result.time_done if result.time_done > 0 else time.time()
        
    # Create a dictionary for the output table
    output = {
        "Time": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": request.get("symbol"),
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        "Price": f"{result.price:.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "SL": f"{request.get('sl'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "TP": f"{request.get('tp'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
    }
    
    # Check if the result includes the position ticket
    if hasattr(result, 'position'):
        output["Position Ticket"] = result.position
    
    return pd.DataFrame([output])


# ==============================================================================
# --- EXECUTION BLOCK ---
# ==============================================================================

# 1. Connect to MT5
if connect_mt5():
    # 2. Get Symbol Information
    symbol_info, pip_value = get_symbol_info(SYMBOL)

    if symbol_info and pip_value:
        # --- PLACE A TRADE HERE ---
        # NOTE: Use mt5.ORDER_TYPE_BUY for a buy order, mt5.ORDER_TYPE_SELL for a sell order.
        
        # Example 1: Place a BUY market order
        trade_result, trade_request = place_market_order(
            symbol_info, 
            pip_value, 
            mt5.ORDER_TYPE_BUY, 
            VOLUME
        )

        # 3. Display the result in a clean table (ideal for Jupyter)
        if trade_result:
            result_df = format_result(trade_result, trade_request)
            print("\n" + "="*50)
            print("--- TRADE EXECUTION RESULT ---")
            print("="*50)
            print(result_df.to_markdown(index=False)) # Display as markdown table in Jupyter
        
        # You can place another trade here if needed, for example a SELL
        # time.sleep(2)
        # trade_result_sell, trade_request_sell = place_market_order(symbol_info, pip_value, mt5.ORDER_TYPE_SELL, VOLUME)
        # if trade_result_sell:
        #     result_df_sell = format_result(trade_result_sell, trade_request_sell)
        #     print("\n--- SECOND TRADE RESULT ---")
        #     print(result_df_sell.to_markdown(index=False))
        

    # 4. Shutdown MT5 connection
    mt5.shutdown()
    print("\nMT5 connection successfully shut down.")

Successfully connected to MT5 terminal.
Account: 40866995

--- Symbol Information for EURUSD ---
Digits: 5
Tick Size (Point): 1e-05
Bid: 1.1549800000000001 | Ask: 1.1550799999999999
None

Sending BUY order for EURUSD at 1.15508...


AttributeError: 'OrderSendResult' object has no attribute 'time_done'

In [70]:
import MetaTrader5 as mt5
import pandas as pd
import time
from datetime import datetime

# --- Configuration ---
SYMBOL = "EURUSD"          # The symbol you wish to trade
VOLUME = 0.01              # The volume of the trade (in lots)
DEVIATION = 10             # Maximum acceptable price deviation from the requested price
MAGIC = 20250615           # A unique identifier for your trades
COMMENT = "Jupyter_Trade_Bot"
SL_OFFSET_PIPS = 30        # Stop Loss offset in pips (e.g., 30 pips)
TP_OFFSET_PIPS = 60        # Take Profit offset in pips (e.g., 60 pips)

def connect_mt5(login=None, password=None, server=None):
    """Initializes and connects to the MT5 terminal."""
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # If credentials are provided, try to log in (useful if terminal is not running)
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
        
    print(f"Successfully connected to MT5 terminal.")
    print(f"Account: {mt5.account_info().login}")
    return True

def get_symbol_info(symbol):
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"{symbol} not found.")
        return None, None

    if not symbol_info.visible:
        print(f"{symbol} is not visible in Market Watch. Attempting to add it...")
        if not mt5.symbol_select(symbol, True):
            print(f"Could not select {symbol}. Trading is not possible.")
            return None, None

    # Get the tick size (e.g., 0.00001 for 5-digit brokers)
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Could not get tick info for {symbol}.")
        return None, None

    # Calculate the value of a single pip for calculations
    # This is typically the 'point' value of the symbol.
    pip_value = symbol_info.point
    
    print(f"\n--- Symbol Information for {symbol} ---")
    print(f"Digits: {symbol_info.digits}")
    print(f"Tick Size (Point): {symbol_info.point}")
    print(print(f"Bid: {tick.bid} | Ask: {tick.ask}"))
    
    return symbol_info, pip_value


def place_market_order(symbol_info, pip_value, order_type, volume):
    """
    Constructs and sends a market order (BUY or SELL).
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    :param volume: Lot size
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        return None

    # Determine execution price and trade type
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS * pip_value)
        tp_price = price + (TP_OFFSET_PIPS * pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS * pip_value)
        tp_price = price - (TP_OFFSET_PIPS * pip_value)
    else:
        print("Invalid order type.")
        return None

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": round(sl_price, symbol_info.digits), # Use symbol digits for rounding
        "tp": round(tp_price, symbol_info.digits), # Use symbol digits for rounding
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, # Good 'Till Cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }

    # Send order to MT5
    print(f"\nSending {type_text} order for {symbol} at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request


def format_result(result, request):
    """Formats the order result and request into a clean pandas DataFrame."""
    if result is None:
        return pd.DataFrame([{"Status": "Failed to send order (local error)"}])
    
    # FIX: Implement a robust check for time attributes, falling back to local time
    # if MT5's time fields (time, time_done) are missing or zero.
    timestamp = time.time() # Default to local time
    
    if hasattr(result, 'time') and result.time > 0:
        timestamp = result.time
    elif hasattr(result, 'time_done') and result.time_done > 0:
        timestamp = result.time_done
        
    # Create a dictionary for the output table
    output = {
        "Time": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": request.get("symbol"),
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        "Price": f"{result.price:.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "SL": f"{request.get('sl'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "TP": f"{request.get('tp'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
    }
    
    # Check if the result includes the position ticket
    if hasattr(result, 'position'):
        output["Position Ticket"] = result.position
    
    return pd.DataFrame([output])


# ==============================================================================
# --- EXECUTION BLOCK ---
# ==============================================================================

# 1. Connect to MT5
if connect_mt5():
    # 2. Get Symbol Information
    symbol_info, pip_value = get_symbol_info(SYMBOL)

    if symbol_info and pip_value:
        # --- PLACE A TRADE HERE ---
        # NOTE: Use mt5.ORDER_TYPE_BUY for a buy order, mt5.ORDER_TYPE_SELL for a sell order.
        
        # Example 1: Place a BUY market order
        trade_result, trade_request = place_market_order(
            symbol_info, 
            pip_value, 
            mt5.ORDER_TYPE_BUY, 
            VOLUME
        )

        # 3. Display the result in a clean table (ideal for Jupyter)
        if trade_result:
            result_df = format_result(trade_result, trade_request)
            print("\n" + "="*50)
            print("--- TRADE EXECUTION RESULT ---")
            print("="*50)
            print(result_df.to_markdown(index=False)) # Display as markdown table in Jupyter
        
        # You can place another trade here if needed, for example a SELL
        # time.sleep(2)
        # trade_result_sell, trade_request_sell = place_market_order(symbol_info, pip_value, mt5.ORDER_TYPE_SELL, VOLUME)
        # if trade_result_sell:
        #     result_df_sell = format_result(trade_result_sell, trade_request_sell)
        #     print("\n--- SECOND TRADE RESULT ---")
        #     print(result_df_sell.to_markdown(index=False))
        

    # 4. Shutdown MT5 connection
    mt5.shutdown()
    print("\nMT5 connection successfully shut down.")

Successfully connected to MT5 terminal.
Account: 40866995

--- Symbol Information for EURUSD ---
Digits: 5
Tick Size (Point): 1e-05
Bid: 1.15523 | Ask: 1.1553499999999999
None

Sending BUY order for EURUSD at 1.15535...

--- TRADE EXECUTION RESULT ---


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [99]:
# --- Configuration ---
#SYMBOL = "EURUSD"          # The symbol you wish to trade
SYMBOL = "Volatility 10 Index"
VOLUME = 1              # The volume of the trade (in lots)
DEVIATION = 20             # Maximum acceptable price deviation from the requested price
MAGIC = 20250615           # A unique identifier for your trades
COMMENT = "Jupyter_Trade_Bot"
SL_OFFSET_PIPS = 500        # Stop Loss offset in pips (e.g., 30 pips)
TP_OFFSET_PIPS = 1000        # Take Profit offset in pips (e.g., 60 pips)

def connect_mt5(login=None, password=None, server=None):
    """Initializes and connects to the MT5 terminal."""
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # If credentials are provided, try to log in (useful if terminal is not running)
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
        
    print(f"Successfully connected to MT5 terminal.")
    print(f"Account: {mt5.account_info().login}")
    return True

def get_symbol_info(symbol):
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"{symbol} not found.")
        return None, None

    if not symbol_info.visible:
        print(f"{symbol} is not visible in Market Watch. Attempting to add it...")
        if not mt5.symbol_select(symbol, True):
            print(f"Could not select {symbol}. Trading is not possible.")
            return None, None

    # Get the tick size (e.g., 0.00001 for 5-digit brokers)
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Could not get tick info for {symbol}.")
        return None, None

    # Calculate the value of a single pip for calculations
    # This is typically the 'point' value of the symbol.
    pip_value = symbol_info.point
    
    print(f"\n--- Symbol Information for {symbol} ---")
    print(f"Digits: {symbol_info.digits}")
    print("pip_value:", pip_value)
    print(f"Tick Size (Point): {symbol_info.point}")
    print(print(f"Bid: {tick.bid} | Ask: {tick.ask}"))
    
    return symbol_info, pip_value


def place_market_order(symbol_info, pip_value, order_type, volume):
    """
    Constructs and sends a market order (BUY or SELL).
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    :param volume: Lot size
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        return None

    # Determine execution price and trade type
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS + pip_value)
        tp_price = price + (TP_OFFSET_PIPS + pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS + pip_value)
        tp_price = price - (TP_OFFSET_PIPS + pip_value)
    else:
        print("Invalid order type.")
        return None

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": round(sl_price, symbol_info.digits), # Use symbol digits for rounding
        "tp": round(tp_price, symbol_info.digits), # Use symbol digits for rounding
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, # Good 'Till Cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }

    # Send order to MT5
    print(f"\nSending {type_text} order for {symbol} at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request


def format_result(result, request):
    """Formats the order result and request into a clean pandas DataFrame."""
    if result is None:
        return pd.DataFrame([{"Status": "Failed to send order (local error)"}])
    
    # Implement a robust check for time attributes, falling back to local time
    timestamp = time.time() # Default to local time
    
    if hasattr(result, 'time') and result.time > 0:
        timestamp = result.time
    elif hasattr(result, 'time_done') and result.time_done > 0:
        timestamp = result.time_done
        
    # Create a dictionary for the output table
    output = {
        "Time": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": request.get("symbol"),
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        "Price": f"{result.price:.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "SL": f"{request.get('sl'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "TP": f"{request.get('tp'):.{mt5.symbol_info(request.get('symbol')).digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
    }
    
    # Check if the result includes the position ticket
    if hasattr(result, 'position'):
        output["Position Ticket"] = result.position
    
    return pd.DataFrame([output])


# ==============================================================================
# --- EXECUTION BLOCK ---
# ==============================================================================

# 1. Connect to MT5
if connect_mt5():
    # 2. Get Symbol Information
    symbol_info, pip_value = get_symbol_info(SYMBOL)

    if symbol_info and pip_value:
        # --- PLACE A TRADE HERE ---
        # NOTE: Use mt5.ORDER_TYPE_BUY for a buy order, mt5.ORDER_TYPE_SELL for a sell order.
        
        # Example 1: Place a BUY market order
        trade_result, trade_request = place_market_order(
            symbol_info, 
            pip_value, 
            mt5.ORDER_TYPE_BUY, 
            VOLUME
        )

        # 3. Display the result in a clean table (ideal for Jupyter)
        if trade_result:
            result_df = format_result(trade_result, trade_request)
            print("\n" + "="*50)
            print("--- TRADE EXECUTION RESULT ---")
            print("="*50)
            # FIX: Switched to .to_string() to avoid the ImportError for missing 'tabulate' dependency.
            print(result_df.to_string(index=False)) 
        
        # You can place another trade here if needed, for example a SELL
        time.sleep(2)
        trade_result_sell, trade_request_sell = place_market_order(symbol_info, pip_value, mt5.ORDER_TYPE_SELL, VOLUME)
        if trade_result_sell:
            result_df_sell = format_result(trade_result_sell, trade_request_sell)
            print("\n--- SECOND TRADE RESULT ---")
            print(result_df_sell.to_string(index=False))
        

    # 4. Shutdown MT5 connection
    mt5.shutdown()
    print("\nMT5 connection successfully shut down.")

Successfully connected to MT5 terminal.
Account: 40866995

--- Symbol Information for Volatility 10 Index ---
Digits: 3
pip_value: 0.001
Tick Size (Point): 0.001
Bid: 5402.045 | Ask: 5402.197
None

Sending BUY order for Volatility 10 Index at 5402.197...

Sending SELL order for Volatility 10 Index at 5402.354...

MT5 connection successfully shut down.


In [120]:
import MetaTrader5 as mt5 # This one works well including calculating optimum lot-size
import pandas as pd
import time
from datetime import datetime
from typing import List

# --- Configuration ---
# List of symbols to trade
SYMBOL_LIST: List[str] = [
    # Volatility Indices
    "Volatility 10 Index","Volatility 25 Index","Volatility 50 Index","Volatility 75 Index","Volatility 100 Index",
    "Volatility 10 (1s) Index","Volatility 25 (1s) Index","Volatility 50 (1s) Index","Volatility 75 (1s) Index","Volatility 100 (1s) Index",
    "Volatility 10 (10s) Index","Volatility 25 (10s) Index","Volatility 50 (10s) Index","Volatility 75 (10s) Index","Volatility 100 (10s) Index",
    # Jump and Step Indices
    "Jump 10 Index","Jump 25 Index","Jump 50 Index","Jump 75 Index","Jump 100 Index",
    "Step Index 25","Step Index 50","Step Index 75","Step Index 100",
    # Major Currency Pairs Added
    "EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCAD", "USDCHF", "NZDUSD", "EURGBP", "EURJPY"
]
# VOLUME constant has been replaced by risk management parameters:
DEVIATION = 10            # Maximum acceptable price deviation
MAGIC = 20250615          # Unique identifier for trades
COMMENT = "Multi_Symbol_Trade_Bot"
SL_OFFSET_PIPS = 10000      # Stop Loss offset in pips
TP_OFFSET_PIPS = 20000      # Take Profit offset in pips

# NEW: Risk Management Configuration
RISK_PERCENTAGE = 0.02    # Risk 1% (0.01) of account equity per trade

# List to collect all trade results
ALL_TRADE_RESULTS = []

def connect_mt5(login=None, password=None, server=None) -> bool:
    """Initializes and connects to the MT5 terminal."""
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # Try to log in if credentials are provided
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
            
    print(f"Successfully connected to MT5 terminal.")
    account_info = mt5.account_info()
    if account_info:
        print(f"Account: {account_info.login}")
    return True

def get_symbol_info(symbol: str):
    """
    Retrieves and prepares the symbol for trading.
    Returns the symbol information object and the price tick value.
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"--- {symbol} NOT FOUND ---")
        return None, None

    if not symbol_info.visible:
        # Check if the symbol can be added to Market Watch
        if not mt5.symbol_select(symbol, True):
            print(f"--- {symbol} NOT TRADABLE: Not visible and could not be added ---")
            return None, None

    # Get the tick size (point value) for SL/TP calculations
    pip_value = symbol_info.point
    
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"--- {symbol} NOT TRADABLE: Could not get tick info ---")
        return None, None
        
    print(f"\n--- Symbol Ready: {symbol} ---")
    print(f"Digits: {symbol_info.digits} | Point: {symbol_info.point} | Bid: {tick.bid} | Ask: {tick.ask}")
    
    return symbol_info, pip_value


def calculate_optimal_volume(symbol: str, entry_price: float, sl_price: float, risk_percent: float) -> float:
    """
    Calculates the optimal lot size based on account equity, risk percentage,
    and the stop loss distance, constrained by symbol limits.

    The formula is: OptimalLotSize = (Equity * RiskPercentage) / (abs(EntryPrice - SLPrice) * ContractSize)
    """
    
    # 1. Get Account Equity
    account_info = mt5.account_info()
    if account_info is None:
        print("Error: Could not retrieve account information.")
        return 0.0
    acc_equity = account_info.balance
    
    # 2. Get Symbol Information for contract size and volume limits
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Error: Could not retrieve symbol info for {symbol}.")
        return 0.0
    
    contract_size = symbol_info.trade_contract_size
    min_volume = symbol_info.volume_min
    max_volume = symbol_info.volume_max
    volume_step = symbol_info.volume_step
    
    # Check for zero values to prevent division by zero
    if contract_size == 0:
        print(f"Error: Contract size is zero for {symbol}. Returning minimum volume.")
        return min_volume
        
    # 3. Calculate Maximum Dollar Loss
    max_loss_dollar = acc_equity * risk_percent
    
    # 4. Calculate Loss per Lot (Loss in price difference * Contract Size)
    price_loss = abs(entry_price - sl_price)
    
    # If SL is too close or equal to entry, use min volume as a fallback.
    if price_loss == 0.0:
        print(f"Warning: SL is equal to entry price for {symbol}. Using minimum volume.")
        return min_volume

    loss_per_lot = price_loss * contract_size
    
    # 5. Calculate Optimal Lot Size (Theoretical)
    optimal_lot_size = max_loss_dollar / loss_per_lot
    
    # 6. Apply Volume Step and Min/Max constraints
    
    # Snap the calculated lot size to the nearest allowed volume step (e.g., 0.01)
    optimal_lot_size = round(optimal_lot_size / volume_step) * volume_step
    
    # Apply min/max volume limits
    final_volume = max(min_volume, min(max_volume, optimal_lot_size))
        
    print(f"  [Risk Calcs] Equity: {acc_equity:.2f}, Risk %: {risk_percent*100}%, Max Loss $: {max_loss_dollar:.2f}, Final Volume: {final_volume:.2f}")
    
    # Ensure the final volume is rounded to the step size's precision
    final_volume = round(final_volume, len(str(volume_step).split('.')[-1]) if '.' in str(volume_step) else 0)

    return final_volume


def place_market_order(symbol_info, pip_value: float, order_type: int):
    """
    Constructs and sends a market order (BUY or SELL) using a dynamically calculated volume.
    
    :param symbol_info: mt5.symbol_info object
    :param pip_value: The size of a single pip (point) for the symbol
    :param order_type: mt5.ORDER_TYPE_BUY or mt5.ORDER_TYPE_SELL
    """
    symbol = symbol_info.name
    
    # Get current price
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"Failed to get current tick for {symbol}.")
        # Return a dictionary indicating local failure before calling format_result
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0} 

    # Determine execution price, trade type, and SL/TP prices
    if order_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        type_text = "BUY"
        # SL/TP calculation (SL below, TP above entry price for BUY)
        sl_price = price - (SL_OFFSET_PIPS * pip_value)
        tp_price = price + (TP_OFFSET_PIPS * pip_value)
    elif order_type == mt5.ORDER_TYPE_SELL:
        price = tick.bid
        type_text = "SELL"
        # SL/TP calculation (SL above, TP below entry price for SELL)
        sl_price = price + (SL_OFFSET_PIPS * pip_value)
        tp_price = price - (TP_OFFSET_PIPS * pip_value)
    else:
        print("Invalid order type.")
        return None, None
        
    # --- DYNAMIC VOLUME CALCULATION ---
    volume_to_use = calculate_optimal_volume(
        symbol, 
        price, 
        sl_price, 
        RISK_PERCENTAGE
    )
    
    if volume_to_use <= 0.0:
        print(f"Skipping trade for {symbol}: Calculated volume is zero or negative.")
        # Return the request dictionary with 0 volume to be logged as a skipped trade
        return None, {"symbol": symbol, "type": order_type, "volume": 0.0}

    # Prepare the request dictionary
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume_to_use, # <-- Dynamic volume used here
        "type": order_type,
        "price": price,
        # Round SL/TP based on the symbol's number of digits
        "sl": round(sl_price, symbol_info.digits), 
        "tp": round(tp_price, symbol_info.digits), 
        "deviation": DEVIATION,
        "magic": MAGIC,
        "comment": COMMENT,
        "type_time": mt5.ORDER_TIME_GTC, 
        "type_filling": mt5.ORDER_FILLING_FOK, 
    }

    # Send order to MT5
    print(f"  Sending {type_text} order for {symbol} with {volume_to_use:.2f} lots at {price:.{symbol_info.digits}f}...")
    result = mt5.order_send(request)

    return result, request


def format_result(result, request):
    """
    Formats the order result and request into a clean dictionary.
    Handles both successful and failed execution attempts.
    """
    symbol = request.get("symbol", "N/A")
    symbol_info = mt5.symbol_info(symbol) if symbol != "N/A" else None
    digits = symbol_info.digits if symbol_info else 4 # Default to 4 digits for formatting
    
    # --- Local Failure Handling (e.g., failed tick check in place_market_order) ---
    if result is None:
        comment_text = request.get("comment", "Local error: Tick or price info failed.")
        if request.get("volume") == 0.0:
            comment_text = "Trade skipped: Volume calculated as zero."
            
        return {
            "Time": datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),
            "Status": "SKIPPED/FAILED (Local)",
            "Return Code": "N/A",
            "Comment": comment_text,
            "Symbol": symbol,
            "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
            "Volume": request.get("volume", "N/A"),
            "Price": "N/A",
            "SL": "N/A",
            "TP": "N/A",
            "Order Ticket": "N/A",
            "Deal Ticket": "N/A",
            "Position Ticket": "N/A",
        }
        
    # --- MT5 Result Handling ---
    # Implement a robust check for time attributes
    timestamp = time.time() # Default to local time
    if hasattr(result, 'time') and result.time > 0:
        timestamp = result.time
    elif hasattr(result, 'time_done') and result.time_done > 0:
        timestamp = result.time_done
        
    # Create the output dictionary
    output = {
        "Time": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "Status": "SUCCESS" if result.retcode == mt5.TRADE_RETCODE_DONE else "FAILURE",
        "Return Code": result.retcode,
        "Comment": result.comment,
        "Symbol": symbol,
        "Type": "BUY" if request.get("type") == mt5.ORDER_TYPE_BUY else "SELL",
        "Volume": request.get("volume"),
        # Format prices using the symbol's digits
        "Price": f"{result.price:.{digits}f}",
        "SL": f"{request.get('sl'):.{digits}f}",
        "TP": f"{request.get('tp'):.{digits}f}",
        "Order Ticket": result.order,
        "Deal Ticket": result.deal,
        "Position Ticket": result.position if hasattr(result, 'position') else "N/A",
    }
    
    return output


# ==============================================================================
# --- MAIN EXECUTION LOGIC ---
# ==============================================================================

def main():
    """Executes the trade bot logic."""
    if connect_mt5():
        
        print("\nStarting multi-symbol trade execution...")
        
        for symbol_name in SYMBOL_LIST:
            # 1. Get Symbol Information
            symbol_info, pip_value = get_symbol_info(symbol_name)
            
            if symbol_info and pip_value:
                # --- PLACE A BUY ORDER ---
                # Note: The 'volume' parameter is omitted here as it's calculated dynamically inside the function.
                trade_result, trade_request = place_market_order(
                    symbol_info, 
                    pip_value, 
                    mt5.ORDER_TYPE_BUY
                )
                
                # 2. Collect the result
                # We check for trade_request to ensure an attempt was made, even if it failed locally
                if trade_request:
                    ALL_TRADE_RESULTS.append(format_result(trade_result, trade_request))
            
            # Add a small delay between trades to respect broker limits
            time.sleep(0.5)

        # 3. Display all results in a single, aggregated DataFrame
        if ALL_TRADE_RESULTS:
            final_df = pd.DataFrame(ALL_TRADE_RESULTS)
            print("\n" + "="*100)
            print("--- AGGREGATED MULTI-SYMBOL TRADE EXECUTION RESULTS (BUY Orders) ---")
            print("="*100)
            # Use max_rows=None to ensure all records are displayed
            pd.set_option('display.max_rows', None) 
            print(final_df.to_string(index=False))
            
        # 4. Shutdown MT5 connection
        mt5.shutdown()
        print("\nMT5 connection successfully shut down.")


if __name__ == "__main__":
    main()

Successfully connected to MT5 terminal.
Account: 40866995

Starting multi-symbol trade execution...

--- Symbol Ready: Volatility 10 Index ---
Digits: 3 | Point: 0.001 | Bid: 5380.293 | Ask: 5380.445
  [Risk Calcs] Equity: 7491.70, Risk %: 2.0%, Max Loss $: 149.83, Final Volume: 14.98
  Sending BUY order for Volatility 10 Index with 14.98 lots at 5380.445...

--- Symbol Ready: Volatility 25 Index ---
Digits: 3 | Point: 0.001 | Bid: 2501.559 | Ask: 2501.741
  [Risk Calcs] Equity: 7491.70, Risk %: 2.0%, Max Loss $: 149.83, Final Volume: 14.98
  Sending BUY order for Volatility 25 Index with 14.98 lots at 2501.741...

--- Symbol Ready: Volatility 50 Index ---
Digits: 4 | Point: 0.0001 | Bid: 141.9465 | Ask: 141.9665
  [Risk Calcs] Equity: 7491.70, Risk %: 2.0%, Max Loss $: 149.83, Final Volume: 149.83
  Sending BUY order for Volatility 50 Index with 149.83 lots at 141.9665...

--- Symbol Ready: Volatility 75 Index ---
Digits: 2 | Point: 0.01 | Bid: 39246.94 | Ask: 39257.6
  [Risk Calcs] E